In [1]:
import cirq
from math import pi

In [2]:
print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)


In [29]:
# Creating a sample circuit using Line Qubits
n_qubits = 5
circuit = cirq.Circuit()
qubits = [cirq.LineQubit(i) for i in range(n_qubits)]

# Applying Hadamard Gate
H = cirq.ParallelGateOperation(cirq.H, qubits)
circuit.append([H])

# Applying CNOT Gate
for i in range(n_qubits - 1):
    circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])], cirq.InsertStrategy.NEW)

# Swapping 0 and 4
SWAP = cirq.SWAP(qubits[0], qubits[4])
circuit.append([SWAP])

# Shifting phase along x axis by pi/2 radians
phase_shift = cirq.rx(pi)
ROTATE = cirq.ParallelGateOperation(phase_shift, qubits)
circuit.append([ROTATE])

# Adding a measurement gate
for q in qubits:
    circuit.append([cirq.measure(q)])

In [30]:
# Printing the circuit
print(circuit)

0: ───H───@───────────────×───Rx(π)───M───
          │               │
1: ───H───X───@───────────┼───Rx(π)───M───
              │           │
2: ───H───────X───@───────┼───Rx(π)───M───
                  │       │
3: ───H───────────X───@───┼───Rx(π)───M───
                      │   │
4: ───H───────────────X───×───Rx(π)───M───


In [31]:
for moment in circuit:
    print(moment)

H(0, 1, 2, 3, 4)
CNOT(0, 1)
CNOT(1, 2)
CNOT(2, 3)
CNOT(3, 4)
SWAP(0, 4)
Rx(π)(0, 1, 2, 3, 4)
cirq.MeasurementGate(1, '0', ())(0) and cirq.MeasurementGate(1, '1', ())(1) and cirq.MeasurementGate(1, '2', ())(2) and cirq.MeasurementGate(1, '3', ())(3) and cirq.MeasurementGate(1, '4', ())(4)


In [32]:
simulator = cirq.Simulator()

In [33]:
result = simulator.run(circuit)
print(result)

0=1
1=0
2=1
3=1
4=1


In [84]:
# Reinitializing for basic lower level testing
qubit = cirq.LineQubit(0)
circuit = cirq.Circuit()

# circuit.append([cirq.H(qubit), cirq.measure(qubit)])
rx_shift = cirq.rx(pi/2)
rz_shift = cirq.rz(pi/2)
circuit.append([rx_shift(qubit), rz_shift(qubit), cirq.measure(qubit)])
print(circuit)

0: ───Rx(0.5π)───Rz(0.5π)───M───


In [95]:
result = simulator.run(circuit)
print(result)

0=1
